In [1]:
import numpy as np
import pickle
import cv2
import os
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation, Flatten, Dropout, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
EPOCHS = 100
BS = 32
default_image_size = tuple((128, 128))
image_size = 0
directory_root = 'D:\Plant-Disease-Detection-main\Plant-Disease-Detection-main\Dataset'
width=128
height=128
depth=3

In [3]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [4]:
image_list, label_list = [], []
try:
  print("[INFO] Loading images....")
  root_dir = listdir(directory_root)
      
  for plant_folder in root_dir:
    print(f"[INFO] Processing {plant_folder} ...")
    plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}")

    for image in plant_disease_image_list[:200]:
      image_directory = f"{directory_root}/{plant_folder}/{image}"
      if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
        image_list.append(convert_image_to_array(image_directory))
        label_list.append(plant_folder)
  print("[INFO] Image loading completed")
except Exception as e:
  print(f"Error : {e}")


[INFO] Loading images....
[INFO] Processing algal leaf in tea ...
[INFO] Processing anthracnose in tea ...
[INFO] Processing Apple Apple scab ...
[INFO] Processing Apple Black rot ...
[INFO] Processing Apple Cedar apple rust ...
[INFO] Processing Apple healthy ...
[INFO] Processing Bacterial leaf blight in rice leaf ...
[INFO] Processing bird eye spot in tea ...
[INFO] Processing Blight in corn Leaf ...
[INFO] Processing Blueberry healthy ...
[INFO] Processing brown blight in tea ...
[INFO] Processing Brown spot in rice leaf ...
[INFO] Processing cabbage looper ...
[INFO] Processing Cercospora leaf spot ...
[INFO] Processing Cherry (including sour) Powdery mildew ...
[INFO] Processing Cherry (including_sour) healthy ...
[INFO] Processing Common Rust in corn Leaf ...
[INFO] Processing Corn (maize) healthy ...
[INFO] Processing corn crop ...
[INFO] Processing Garlic ...
[INFO] Processing ginger ...
[INFO] Processing Grape Black rot ...
[INFO] Processing Grape Esca Black Measles ...
[INFO

In [5]:
image_size = len(image_list)

In [6]:
image_size

10217

In [7]:
label_list

['algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal leaf in tea',
 'algal le

Transform Image Labels uisng Scikit Learn's LabelBinarizer

In [8]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer, open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [9]:
print(label_binarizer.classes_)

['Apple Apple scab' 'Apple Black rot' 'Apple Cedar apple rust'
 'Apple healthy' 'Bacterial leaf blight in rice leaf'
 'Blight in corn Leaf' 'Blueberry healthy' 'Brown spot in rice leaf'
 'Cercospora leaf spot' 'Cherry (including sour) Powdery mildew'
 'Cherry (including_sour) healthy' 'Common Rust in corn Leaf'
 'Corn (maize) healthy' 'Garlic' 'Grape Black rot'
 'Grape Esca Black Measles' 'Grape Leaf blight Isariopsis Leaf Spot'
 'Grape healthy' 'Gray Leaf Spot in corn Leaf' 'Leaf smut in rice leaf'
 'Nitrogen deficiency in plant' 'Orange Haunglongbing Citrus greening'
 'Peach healthy' 'Pepper bell Bacterial spot' 'Pepper bell healthy'
 'Potato Early blight' 'Potato Late blight' 'Potato healthy'
 'Raspberry healthy' 'Sogatella rice' 'Soybean healthy'
 'Strawberry Leaf scorch' 'Strawberry healthy' 'Tomato Bacterial spot'
 'Tomato Early blight' 'Tomato Late blight' 'Tomato Leaf Mold'
 'Tomato Septoria leaf spot' 'Tomato Spider mites Two spotted spider mite'
 'Tomato Target Spot' 'Tomato 

In [10]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0

Splitting the Data into train data and test data

In [11]:
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state=42)

In [12]:
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, horizontal_flip=True,
                         fill_mode="nearest")

__Model Creation__

In [13]:
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format == "channels_first":
  inputShape = (depth, height, width)
  chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same", input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 128, 128, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 42, 42, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 42, 42, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 42, 42, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 42, 42, 64)        0

In [15]:
opt = Adam(learning_rate=0.001, decay = 0.001 / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer = opt, metrics = ["accuracy"])

In [16]:
history = model.fit(aug.flow(x_train, y_train, batch_size=BS), validation_data=(x_test, y_test),
                              epochs = EPOCHS)

Epoch 1/100
256/256 [==============================] - 130s 423ms/step - loss: 2.7178 - accuracy: 0.3192 - val_loss: 12.6375 - val_accuracy: 0.0357
Epoch 2/100
256/256 [==============================] - 101s 394ms/step - loss: 1.8658 - accuracy: 0.4893 - val_loss: 2.6998 - val_accuracy: 0.3973
Epoch 3/100
256/256 [==============================] - 106s 416ms/step - loss: 1.5013 - accuracy: 0.5719 - val_loss: 2.6988 - val_accuracy: 0.4173
Epoch 4/100
256/256 [==============================] - 109s 424ms/step - loss: 1.2700 - accuracy: 0.6181 - val_loss: 1.5229 - val_accuracy: 0.5925
Epoch 5/100
256/256 [==============================] - 103s 404ms/step - loss: 1.1377 - accuracy: 0.6601 - val_loss: 2.0475 - val_accuracy: 0.5093
Epoch 6/100
256/256 [==============================] - 100s 390ms/step - loss: 1.0016 - accuracy: 0.6961 - val_loss: 2.7725 - val_accuracy: 0.4501
Epoch 7/100
256/256 [==============================] - 103s 401ms/step - loss: 0.8813 - accuracy: 0.7238 - val_loss: 